# Built Area - Atlas of Urban Expansion
### (Original Atlas of Urban Expansion Esque Output)

### Start Here:

In [ ]:
#  noqa

from pathlib import Path
import geopandas as gpd
import pandas as pd
import numpy as np  # noqa
import rasterio  # noqa
from rasterio import features  # noqa
import os
import shutil  # noqa
import sys

pd.options.mode.copy_on_write = True

module_path = str(Path(os.getcwd()).parents[1])
if module_path not in sys.path:
    sys.path.append(module_path)
os.chdir(module_path)

In [ ]:
from IO.Scripts.Modules.rasters import (  # noqa
    vec_2_rast,
    region_group,
    zonal_stats,
    raster_out,
)
from IO.Scripts.Modules.setup import (  # noqa
    folder_set_up,
    dwnld_import,
    create_extents,
    wgs84_to_utm,
    alt_poly,
)
from IO.Scripts.Modules.vectors import filt_load_bldgs  # noqa
from IO.Scripts.Modules.terrain import get_terrain_data  # noqa

In [ ]:
# INPUTS
cityid = 1717
out_path = Path(r"C:\Users\MAtkinson\Documents\GitHub\urbex\IO\Data")
release = "2025-09-24.0"
overture_location = "s3://overturemaps-us-west-2"
alt = {
    "fp": out_path / r"Rwanda/Kigali/Google/wb_KK_polygons/Kigali_Kamonyi.shp",
    "id": "597000",
    "city": "Kigali",
    "country": "Rwanda",
}  # or None

In [ ]:
# Download city shape, name, and country
if isinstance(alt, dict):
    uc = alt_poly(fp=alt["fp"], id=alt["id"], city=alt["city"], country=alt["country"])
else:
    uc = dwnld_import(cityid, out_path)

# Create extent data and projected crs from city shape
extentPoly, zoom, bounds, xmin, xmax, ymin, ymax = create_extents(uc.geometry)
UTMZone, wkid = wgs84_to_utm(uc.geometry)

# extract city and country name
city_name = uc["GC_UCN_MAI_2025"].tolist()[0]
country_name = uc["GC_CNT_GAD_2025"].tolist()[0]

# set up folder structure
outfp, fdict = folder_set_up(out_path, city_name, country_name)
Path(outfp, "Current_Urban_Extent").mkdir(parents=True, exist_ok=True)
fdict["cue"] = outfp / "Current_Urban_Extent"
print(fdict["cue"])

In [ ]:
bldgs = f"{city_name}_bldg_big.shp"
sampTIF = f"{city_name}_elevation.tif"

bldgTIF = f"{city_name}_bldg.tif"
bldgREG = f"{city_name}_bldg_regions.tif"

### Move or Download Data:

In [ ]:
# Elevation - Move or Download
if not Path(fdict["cue"], sampTIF).exists():
    if Path(fdict["Model_Inputs"], sampTIF).exists():
        # copy and move
        shutil.copy(Path(fdict["Model_Inputs"], sampTIF), Path(fdict["cue"], sampTIF))
    else:
        # download
        get_terrain_data(bounds, zoom, fdict["cue"], city_name)

In [ ]:
# Big Buildings - Move or Download
if not Path(fdict["cue"], bldgs).exists():
    if Path(fdict["Downloads"], bldgs).exists():
        # copy and move
        shutil.copy(Path(fdict["Downloads"], bldgs), Path(fdict["cue"], bldgs))
    else:
        # buildings
        filt_load_bldgs(
            dloc=(
                f"{overture_location}/release/{release}/"
                "theme=buildings/type=building/*"
            ),
            big_out=str(fdict["cue"] / bldgs),
            samp_out=str(fdict["cue"] / f"{city_name}_bldg_samp_pts.shp"),
            pt_out=None,
            cols="subtype, class, height, num_floors, geometry",
            xmin=xmin,
            xmax=xmax,
            ymin=ymin,
            ymax=ymax,
        )

### Calculate Built Area Metrics:

In [ ]:
# set working directory
os.chdir(fdict["cue"])

In [ ]:
# buffer buildings by 5m
bldg_buf = gpd.read_file(bldgs).to_crs(wkid)
bldg_buf["geometry"] = (
    bldg_buf["geometry"].buffer(5, join_style="mitre", cap_style="square").to_crs(4326)
)
# building polygons to raster
bldgrast = vec_2_rast(bldg_buf, sampTIF, bldgTIF)

In [ ]:
# region group
rg_out = fdict["cue"] / f"{city_name}_bldg_reg_grp.tif"
r = region_group(bldgTIF, rg_out, bldgTIF, n=8, nv=-9999)

In [ ]:
# count area of each group and set to value of raster region
# zonal statistics
zsa, zonal_tab = zonal_stats(zones_fp=rg_out, values_fp=bldgTIF, sf="count")
# raster calculator
zsa[zsa < 1500] = -9999

In [ ]:
zonal_output = fdict["cue"] / f"{city_name}_big_zone_urb.tif"
raster_out(rast=zsa, ofp=zonal_output, exfp=bldgTIF)